# Method 3
## Ready-made Foreground without Background

In [1]:
import os
import random
import numpy as np
from PIL import Image

### Image Composition
- Takes in a foreground image path and a background image path
- Randomly rotates and scales the foreground
- Pastes the foreground on the background
- Creates a bit-mask image
- Calculates a bounding box
- Returns the image, mask, and bounding box

In [2]:
def compose_images(foreground_path, background_path):
    # Make sure the foreground path is valid and open the image
    foreground = Image.open(foreground_path)

    # Make sure the background path is valid and open the image
    background = Image.open(background_path)
    background = background.convert('RGBA')
    
    # Rotate the foreground
    angle_degrees = random.randint(0, 45)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True)
    
    # Scale the foreground
    scale = random.random() * .5 + .5 # Pick something between .5 and 1
    new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
    foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    
    # Choose a random x,y position for the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    assert max_xy_position[0] >= 0 and max_xy_position[1] >= 0, \
        'foreground {} is to big for the background {}'.format(foreground_path, background_path)
    paste_position = (random.randint(0, max_xy_position[0]), random.randint(0, max_xy_position[1]))
    
    # Create a new foreground image as large as the background and paste it on top
    new_foreground = Image.new('RGBA', background.size, color = (0, 0, 0, 0))
    new_foreground.paste(foreground, paste_position)
        
    # Extract the alpha channel from the foreground and paste it into a new image the size of the background
    alpha_mask = foreground.getchannel(3)
    new_alpha_mask = Image.new('L', background.size, color=0)
    new_alpha_mask.paste(alpha_mask, paste_position)
    
    composite = Image.composite(new_foreground, background, new_alpha_mask)
    
    # Grab the alpha pixels above a specified threshold
    alpha_threshold = 200
    mask_arr = np.array(np.greater(np.array(new_alpha_mask), alpha_threshold), dtype=np.uint8)
    hard_mask = Image.fromarray(np.uint8(mask_arr) * 255, 'L')
    
    # Get the smallest & largest non-zero values in each dimension and calculate the bounding box
    nz = np.nonzero(hard_mask)
    bbox = [np.min(nz[0]), np.min(nz[1]), np.max(nz[0]), np.max(nz[1])] 

    return composite, hard_mask, bbox
    

### Image Generation
- Get the image paths
- Generate new images

In [11]:
# Get lists of foreground and background image paths
dataset_dir = 'data3'
backgrounds_dir = os.path.join(dataset_dir, 'input/background/cloister')
foregrounds_dir = os.path.join(dataset_dir, 'input/foreground')
backgrounds = [os.path.join(backgrounds_dir, file_name) for file_name in os.listdir(backgrounds_dir)]
foregrounds = [os.path.join(foregrounds_dir, file_name) for file_name in os.listdir(foregrounds_dir)]

# Output directory
output_dir = os.path.join(dataset_dir, 'output')


# Generate new images
for i in range(5):
    foreground_path = random.choice(foregrounds)
    background_path = random.choice(backgrounds)
    if (background_path == os.path.join(backgrounds_dir,'.DS_Store') or
        foreground_path == os.path.join(foregrounds_dir,'.DS_Store')):
        continue
    composite, mask, bbox = compose_images(foreground_path, background_path)
    
    composite_path = os.path.join(output_dir, 'image/{}.png'.format(i))
    composite.save(composite_path)
    
    mask_path = os.path.join(output_dir, 'mask/{}.png'.format(i))
    mask.save(mask_path)
    

#### Code Reference:
https://github.com/akTwelve/tutorials/blob/master/image_composition/BasicImageComposition.ipynb